In [1]:
import pandas as pd
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)
from nltk.corpus import stopwords
import re as re
from pyspark.ml.feature import CountVectorizer , IDF
from pyspark.mllib.linalg import Vector, Vectors
#from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.ml.clustering import LDA, LDAModel
import nltk
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import size
from pyspark.sql.functions import udf, col
from pyspark.ml.feature import RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import udf, struct
import pyspark.sql.types as T 
import string
import nltk
from pyspark.sql import Row

import os
import sys
from pyspark.sql import SparkSession



In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark =SparkSession.builder.master("local[1]").appName('SparkProject').getOrCreate() 

21/12/10 09:10:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/10 09:10:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#data  = spark.read.json(r"C:\Users\EndUser\Desktop\Data\AbhishekData\tweets.json")
#df = spark.read.json("tweets2.csv")
df = spark.read.json("tweets90.json")

In [4]:
df.head()

Row(content='.@VW will introduce heads-up display with augmented-reality in its upcoming #EVs. https://t.co/CINqosgiCD', date='2017-01-31T23:49:34+00:00', latitude=None, longitude=None, tweetid=826578200402616320, userid=1187313746, userlocation='')

In [5]:
df = df.dropDuplicates(['tweetid'])
df = df.na.drop()
df = df.filter("content != ''")
df = df.limit(100)

In [6]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_number(tweet):
    tweet = re.sub('([0-9]+)', '', tweet) 
    return tweet

def remove_users(tweet):
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    return tweet

In [7]:
from pyspark.sql.types import StringType

remove_emoji=udf(remove_emoji)
remove_users=udf(remove_users)
remove_number=udf(remove_number)

df=df.withColumn('content', remove_emoji(df['content']))
df=df.withColumn('content', remove_users(df['content']))
df=df.withColumn('content', remove_number(df['content']))

In [8]:

df = df.withColumn('content', F.regexp_replace('content', r'http\S+', ''))
df = df.withColumn('content', F.regexp_replace('content', '@\w+', ''))
df = df.withColumn('content', F.regexp_replace('content', '#', ''))
df = df.withColumn('content', F.regexp_replace('content', 'RT', ''))
df = df.withColumn('content', F.regexp_replace('content', ':', ''))
df = df.withColumn('content', F.regexp_replace('content', '\n', ''))
df = df.withColumn('content', F.regexp_replace('content', '|', ''))
df = df.withColumn('content', F.regexp_replace('content', '[^\sa-zA-Z0-9]', ''))
df = df.withColumn("content",F.lower('content'))
df = df.withColumn("content",F.rtrim('content'))

In [9]:
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ')

In [10]:
tokenizer = RegexTokenizer().setPattern("[\\W_]+").setMinTokenLength(3).setInputCol("content").setOutputCol("tokens")

In [11]:
df = tokenizer.transform(df)


In [12]:
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ', tokens=['european', 'fast', 'charging', 'networks', 'announce', 'fast', 'charging', 'alliance', 'evs'])

In [13]:
nltk.download('stopwords')
stopwordList = stopwords.words('english')
remover = StopWordsRemover(stopWords=stopwordList)
remover.setInputCol("tokens")
remover.setOutputCol("filtered")

df = remover.transform(df);


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bigdata/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatization(row):
    #row = [stemmer.stem(lemmatizer.lemmatize(word)) for word in row]
    row = [lemmatizer.lemmatize(word,'v') for word in row]
    return row

[nltk_data] Downloading package wordnet to /home/bigdata/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
lemmatization = udf(lemmatization)

df=df.withColumn('tokens', lemmatization(df['filtered']))



In [16]:
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ', tokens='[european, fast, charge, network, announce, fast, charge, alliance, evs]', filtered=['european', 'fast', 'charging', 'networks', 'announce', 'fast', 'charging', 'alliance', 'evs'])

In [17]:
#sampling data for manual labelling
def get_sample(part=0.17):
    df = df.sample(part)
    df = df.select("tweetid", "content")
    df.coalesce(1).write.option("header", True).csv('sample.csv')

In [18]:
#training and topic modelling

from pyspark.sql.functions import lit
df = df.withColumn("polarity", lit(0))
df = df.withColumn("topic", lit(""))

In [19]:
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ', tokens='[european, fast, charge, network, announce, fast, charge, alliance, evs]', filtered=['european', 'fast', 'charging', 'networks', 'announce', 'fast', 'charging', 'alliance', 'evs'])

In [45]:
#extracting tweet sentiment

In [20]:
from textblob import TextBlob

sentiment_polarity = udf(lambda x: TextBlob(x).sentiment[0])
spark.udf.register("sentiment_polarity", sentiment_polarity)

sentiment_subjectivity = udf(lambda x: TextBlob(x).sentiment[1])
spark.udf.register("sentiment_subjectivity", sentiment_subjectivity)

<function __main__.<lambda>(x)>

In [21]:
df = df.withColumn('sentiment_polarity',sentiment_polarity('content').cast('double'))
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ', tokens='[european, fast, charge, network, announce, fast, charge, alliance, evs]', filtered=['european', 'fast', 'charging', 'networks', 'announce', 'fast', 'charging', 'alliance', 'evs'], sentiment_polarity=0.13333333333333333)

In [22]:
df = df.withColumn('sentiment_subjectivity',sentiment_subjectivity('content').cast('double'))
df.head()

Row(content=' european fast charging networks announce fast charging alliance evs', date='2017-02-12T19:18:39+00:00', latitude=59.8097794, longitude=10.4914682, tweetid=830858673421443077, userid=1664893472, userlocation='📍Amsterdam & Costa Rica ', tokens='[european, fast, charge, network, announce, fast, charge, alliance, evs]', filtered=['european', 'fast', 'charging', 'networks', 'announce', 'fast', 'charging', 'alliance', 'evs'], sentiment_polarity=0.13333333333333333, sentiment_subjectivity=0.39999999999999997)

In [23]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [24]:
#Collecting the data from spark and save to list
words_array = df.select("filtered").rdd.flatMap(lambda x: x).collect()
txt = df.select("content").rdd.flatMap(lambda x: x).collect()

In [25]:
id2word = corpora.Dictionary(words_array)

In [26]:
texts = words_array
corpus = [id2word.doc2bow(text) for text in texts]
bow = corpus

In [27]:
#printing the bag of words
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('alliance', 1),
  ('announce', 1),
  ('charging', 2),
  ('european', 1),
  ('evs', 1),
  ('fast', 2),
  ('networks', 1)]]

In [28]:
#function to create and run the guided lda model
def test_eta(eta, dictionary, ntopics, print_topics=True, print_dist=True):
    corp = words_array
    np.random.seed(42) # set the random seed for repeatability
    bow = [dictionary.doc2bow(line) for line in corp] # get the bow-format lines with the set dictionary
    with (np.errstate(divide='ignore')):  # ignore divide-by-zero warnings
        model = gensim.models.ldamodel.LdaModel(
            corpus=bow, id2word=dictionary, num_topics=ntopics,
            random_state=42, chunksize=100, eta=eta,
            eval_every=-1, update_every=1,
            passes=150, alpha='auto', per_word_topics=True)
    # visuzlize the model term topics
    # viz_model(model, dictionary)
    print('Perplexity: {:.2f}'.format(model.log_perplexity(bow)))
    if print_topics:
        # display the top terms for each topic
        for topic in range(ntopics):
            print('Topic {}: {}'.format(topic, [dictionary[w] for w,p in model.get_topic_terms(topic, topn=3)]))
    if print_dist:
        # display the topic probabilities for each document
        for line,bag in zip(txt,bow):
            doc_topics = ['({}, {:.1%})'.format(topic, prob) for topic,prob in model.get_document_topics(bag)]
            print('{} {}'.format(line, doc_topics))
    return model

In [29]:
#function to create the eta for topic modelling
def create_eta(priors, etadict, ntopics):
    eta = np.full(shape=(ntopics, len(etadict)), fill_value=1) # create a (ntopics, nterms) matrix and fill with 1
    for word, topic in priors.items(): # for each word in the list of priors
        keyindex = [index for index,term in etadict.items() if term==word] # look up the word in the dictionary
        if (len(keyindex)>0): # if it's in the dictionary
            print(keyindex)
            eta[topic,keyindex[0]] = 1e7  # put a large number in there
    eta = np.divide(eta, eta.sum(axis=0)) # normalize so that the probabilities sum to 1 over all topics
    print (eta)
    return eta

In [30]:
topic_labelling = {
    'cost': 0,'value': 0,'worth': 0,'valuation': 0,'premium': 0,'rate': 0,'overprice': 0,'cheap': 0,'discount': 0,'inflation': 0,'pay': 0,'amount': 0,'expense': 0,'expensive': 0,'dollar': 0,'money': 0,'costly': 0,'prise': 0,'priceless': 0,'undervalue': 0,'overvalue': 0,'revaluation': 0,'pennyworth': 0,'economy': 0,'underrate': 0,'underprice': 0,'profit': 0,'estimate':0 ,'fees':0 ,'marginal':0 ,'cash':0 ,'pocket':0 ,'capital':0 ,'dime':0 ,'markup':0 ,'haggle':0 ,'wholesale':0 ,'quotation':0 ,'stipend':0 ,'afford':0 ,'savings':0 ,'save':0,'range':1 ,'anxiety':1 ,'far':1 ,'distance':1 ,'long':1 ,'area':1 ,'long-range':1 ,'short-range':1 ,'travel':1 ,'near':1 ,'mileage':1 ,'afar':1 ,'farther':1 ,'reach':1 ,'remote':1 ,'short':1 ,'stretch':1 ,'haul':1 ,'distant':1 ,'commute':1 ,'extended-range':1,'fuel':2 ,'zero':2 ,'emission':2 ,'das':2 ,'diesel':2 ,'refuel':2 ,'coal':2 ,'oil ':2 ,'charcoal':2 ,'biogas':2 ,'fossil':2 ,'electric':2 ,'ev':2,'eco':3 ,'eco-friendly':3 ,'carbon':3 ,'footprint':3 ,'pollution':3 ,'sustainable':3 ,'renewal':3 ,'power':3 ,'plug':3 ,'green':3 ,'environment':3 ,'earth':3 ,'friendly':3 ,'carbon':3 ,'footprint':3,'batttery':4 ,'cell':4 ,'life':4 ,'charge':4 ,'charger':4 ,'voltage':4 ,'elctrolyte':4 ,'surcharge':4 ,'lithium-ion':4 ,'lithium':4 ,'ion':4 ,'battery cost':4 ,'battery life':4 ,'Fuel cell':4 ,'killowatt':4 ,'hour':4 ,'killowatt-hour':4 ,'regenerative':4 ,'regenerative-braking':4, 'infra':5 ,'infrastructure':5 ,'stations':5 ,'charging':5 ,'plug':5 ,'chademo':5 ,'fast charging':5 ,'supercharger':5 ,'framework':5 ,'structure':5 ,'groundwork':5 ,'support':5
}
eta = create_eta(topic_labelling, id2word, 6)
lda_model = test_eta(eta, id2word, 6,False,False)

[492]
[844]
[457]
[516]
[203]
[296]
[748]
[304]
[744]
[348]
[318]
[120]
[555]
[100]
[131]
[557]
[317]
[491]
[459]
[326]
[268]
[624]
[2]
[276]
[[1.66666667e-01 1.66666667e-01 9.99999500e-08 ... 1.66666667e-01
  1.66666667e-01 1.66666667e-01]
 [1.66666667e-01 1.66666667e-01 9.99999500e-08 ... 1.66666667e-01
  1.66666667e-01 1.66666667e-01]
 [1.66666667e-01 1.66666667e-01 9.99999500e-08 ... 1.66666667e-01
  1.66666667e-01 1.66666667e-01]
 [1.66666667e-01 1.66666667e-01 9.99999500e-08 ... 1.66666667e-01
  1.66666667e-01 1.66666667e-01]
 [1.66666667e-01 1.66666667e-01 9.99999500e-08 ... 1.66666667e-01
  1.66666667e-01 1.66666667e-01]
 [1.66666667e-01 1.66666667e-01 9.99999500e-01 ... 1.66666667e-01
  1.66666667e-01 1.66666667e-01]]
Perplexity: 11.74
